In [ ]:
from bs4 import BeautifulSoup
import re
import mysql.connector
from mysql.connector import Error
import requests
# from Ipython.display import clear_output

In [63]:
# MySQL connection configs
DB_HOST = '127.0.0.1'
DB_USER = 'test_user' 
DB_PASSWORD = 'StrongPass123!'
DB_NAME = 'scraping'
TBL_NAME = 'countries'

In [64]:

config = {
  'user': DB_USER,
  'password': DB_PASSWORD,
  'host': DB_HOST,
  'raise_on_warnings': True
}

In [5]:
def check_database_existance(connection):
    mycursor = connection.cursor(buffered = True)
    mycursor.execute("SHOW DATABASES")
    
    for db in mycursor:
        if db[0] == DB_NAME:
            return True
    return False

def check_table_exist(connection):
    mycursor = connection.cursor(buffered = True)
    mycursor.execute("SHOW TABLES")
    
    for tbl in mycursor:
        if tbl[0] == TBL_NAME:
            return True
    
    return False


In [51]:
def init_mysql():
    # Choosing db and connect to it
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor(buffered= True)
    # Creating database if not exist
    if not check_database_existance(conn):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
    
    conn.database = DB_NAME
    # Creating table if not exist
    if not check_table_exist(conn):
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {TBL_NAME} (
                country VARCHAR(255) PRIMARY KEY,
                capital VARCHAR(255) NOT NULL,
                population INT,
                area DECIMAL(10,1)
            )
        """)
    conn.commit()
    return conn


In [41]:
url = "https://www.scrapethissite.com/pages/simple/"
req = requests.get(url)
print(req)

<Response [200]>


In [42]:
soup = BeautifulSoup(req.text, 'html.parser')

In [43]:
countries = soup.find_all('div', attrs= {'class' : 'col-md-4 country'})
records = []
for country in countries:
    country_name = country.find_all('h3', attrs= {'class' : 'country-name'})
    country_capital = country.find_all('span', attrs= {'class' : 'country-capital'})
    country_population = country.find_all('span', attrs= {'class' : 'country-population'})
    country_area = country.find_all('span', attrs= {'class' : 'country-area'})
    country_name_text = (re.sub(r'\s', ' ', country_name[0].text)).strip()
    country_capital_text = (re.sub(r'\s', ' ', country_capital[0].text)).strip()
    country_population_text = (re.sub(r'\s', ' ', country_population[0].text)).strip()
    country_area_text = (re.sub(r'\s', ' ', country_area[0].text)).strip()
    record = (country_name_text, country_capital_text, country_population_text, country_area_text)
    records.append(record)

In [44]:
def convert_to_decimal_one(s):

    x = float(s)
    return round(x, 1)

def convert_to_integer(s):
    
    x = int(s)
    return x

In [66]:
try:
    conn = init_mysql()
    cursor = conn.cursor(buffered=True)
    insert_query = '''INSERT INTO countries (country, capital, population, area)
                    VALUES (%s, %s, %s, %s)
                    '''
    n = 0
    for name, capital, population, area in records:
        if n < 20:
            try:
                population = convert_to_integer(population)
            except ValueError:
                print(f'Wrong value for {name} population: {population}')
                
            try:
                area = convert_to_decimal_one(area)
            except ValueError:
                print(f'Wrong value for {name} area: {area}')

            try:
                cursor.execute(insert_query, (name, capital, population, area))
                n += 1
            except Error as e:
                if e.errno == 1062 and "Duplicate entry" in str(e):
                    # clear_output()
                    print('Country already exist')
                    n += 1
                else:
                    print(f'Error in {name}: {e}')
        else:
            break
except Error as conn_e:
    print('Connection to MySQL error:', conn_e)
finally:
    if conn.is_connected():
        conn.commit()
        cursor.close()
        conn.close()
        print('Connection to database closed')

Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Country already exist
Connection to database closed
